In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time
import re

In [3]:
test_set = pd.read_csv("/content/drive/MyDrive/STKI/uts/testdata.csv",encoding="latin1")
df_train = pd.read_csv('/content/drive/MyDrive/STKI/uts/traindata.csv')
df_train.head()

,Id,Sentiment,Pasangan Calon,Text Tweet
0,1,negative,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...
1,2,negative,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,negative,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,4,negative,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...
4,5,negative,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


In [4]:
df_train.drop('Id', axis=1, inplace=True)

In [5]:
df_train.drop('Pasangan Calon', axis=1, inplace=True)

In [6]:
df_train.head()

,Sentiment,Text Tweet
0,negative,Banyak akun kloning seolah2 pendukung #agussil...
1,negative,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,negative,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,negative,Kasian oh kasian dengan peluru 1milyar untuk t...
4,negative,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


In [7]:
df_train['Text Tweet'] = df_train['Text Tweet'].str.lower()


print('Case Folding Result : \n')

print(df_train['Text Tweet'].head(5))
print('\n\n\n')

Case Folding Result : 

0    banyak akun kloning seolah2 pendukung #agussil...
1    #agussilvy bicara apa kasihan yaa...lap itu ai...
2    kalau aku sih gak nunggu hasil akhir qc tp lag...
3    kasian oh kasian dengan peluru 1milyar untuk t...
4    maaf ya pendukung #agussilvy..hayo dukung #ani...
Name: Text Tweet, dtype: object






In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df_train['Text Tweet'] = df_train['Text Tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df_train['Text Tweet'] = df_train['Text Tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df_train['Text Tweet'] = df_train['Text Tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df_train['Text Tweet'] = df_train['Text Tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df_train['Text Tweet'] = df_train['Text Tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df_train['Text Tweet'] = df_train['Text Tweet'].apply(remove_singl_char)

# NLTK word rokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df_train['tweet_tokens'] = df_train['Text Tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(df_train['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [banyak, akun, kloning, seolah, pendukung, mul...
1    [bicara, apa, kasihan, yaalap, itu, air, matan...
2    [kalau, aku, sih, gak, nunggu, hasil, akhir, q...
3    [kasian, oh, kasian, dengan, peluru, milyar, u...
4       [maaf, ya, pendukung, hayo, dukung, diputaran]
Name: tweet_tokens, dtype: object






In [59]:
df_train.head()

,Sentiment,Text Tweet,tweet_tokens
0,negative,banyak akun kloning seolah pendukung mulai men...,"[banyak, akun, kloning, seolah, pendukung, mul..."
1,negative,bicara apa kasihan yaalap itu air matanya wkwk...,"[bicara, apa, kasihan, yaalap, itu, air, matan..."
2,negative,kalau aku sih gak nunggu hasil akhir qc tp lag...,"[kalau, aku, sih, gak, nunggu, hasil, akhir, q..."
3,negative,kasian oh kasian dengan peluru milyar untuk ti...,"[kasian, oh, kasian, dengan, peluru, milyar, u..."
4,negative,maaf ya pendukung hayo dukung diputaran,"[maaf, ya, pendukung, hayo, dukung, diputaran]"


In [85]:
from sklearn.model_selection import train_test_split

X, y = df_train.drop(columns=['Sentiment']), df_train['Sentiment']
print(X.shape)
print(y.shape)
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, stratify=y)

(750, 2)
(750,)


In [86]:
train_counts = y_train.value_counts().sort_index()
val_counts = y_validate.value_counts().sort_index()

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

cv = CountVectorizer()
token_counts = cv.fit_transform(X_train['Text Tweet'])

In [88]:
X_train = pd.DataFrame.sparse.from_spmatrix(token_counts, columns=cv.get_feature_names_out())
X_train.head()

,aa,aamiin,abal,abang,abis,absurd,acara,acaranya,ada,adalah,...,yangtelahmati,ye,yg,yme,yra,yudhoyono,yuk,yusuf,zalim,zippermouth
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
from sklearn.naive_bayes import MultinomialNB

nbc = MultinomialNB()
nbc.fit(X_train, y_train)

MultinomialNB()

In [90]:
val_counts = cv.transform(X_validate['Text Tweet'])

In [91]:
X_validate = pd.DataFrame.sparse.from_spmatrix(val_counts, columns=cv.get_feature_names_out())
y_pred = nbc.predict(X_validate)

In [92]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def print_performance(clf, X, y):
    y_pred = clf.predict(X)

    accuracy = accuracy_score(y_pred, y)
    print("Accuracy: ", accuracy)
    print("Precision:", precision_score(y_pred, y, average='weighted'))
    print("Recall:   ", recall_score(y_pred, y, average='weighted'))
    print("F1 Score: ", f1_score(y_pred, y, average='weighted'))

    cm = confusion_matrix(y, y_pred)

In [93]:
cv = CountVectorizer()

token_counts = cv.fit_transform(X['Text Tweet'])
X = pd.DataFrame.sparse.from_spmatrix(token_counts, columns=cv.get_feature_names_out())

clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB()

In [94]:
clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB()

In [95]:
X_test = pd.read_csv("/content/drive/MyDrive/STKI/uts/testdata.csv")
id = X_test['Id']
test_counts = cv.transform(X_test['Text Tweet'].values.astype('U'))
X_test = pd.DataFrame.sparse.from_spmatrix(test_counts, columns=cv.get_feature_names_out())
y_pred = clf.predict(X_test)

In [96]:
df_hasil = pd.DataFrame({
    'Id': id,
    'Category': y_pred
})
df_hasil.head()

,Id,Category
0,807,positive
1,189,positive
2,223,positive
3,408,negative
4,8,positive


In [97]:
df_hasil.to_csv("hasilhasil")

In [98]:
df_hasil.head()

,Id,Category
0,807,positive
1,189,positive
2,223,positive
3,408,negative
4,8,positive
